In [1]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os

# Data manipulation and visualization
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

# Modeling algorithms
# General
import statsmodels.api as sm
from scipy import stats

# Model selection
from sklearn.model_selection import train_test_split

# Evaluation metrics
from sklearn import metrics
# for classification
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# 한글 패치
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font)

# 결측값 시각화 모듈
import missingno as msno

In [2]:
# 종속변수 설정 (0 : 참여안함, 1 : 참여함)
def Y_variable(df):
    # 비형식교육 참여여부 생성
    df['비형식교육 참여여부'] = df['비형식교육 참여자'].map({1: 1, 2: 0})
    df = df.drop(['비형식교육 참여자'], axis=1)
    return df

In [3]:
# 형식, 비형식 유형별 컬럼 재분류 함수
def reclassify_program(x):
    if 1 <= x <= 10:
        return 1
    elif x == 11:
        return 2
    elif 12 <= x <= 17:
        return 3
    elif 18 <= x <= 24:
        return 4
    elif 25 <= x <= 27:
        return 5
    elif 28 <= x <= 30:
        return 6
    else:
        return np.nan

In [4]:
# 파생변수 생성 함수
def derived_variable(df):
    # 파생변수 변환 컬럼 선택
    col_groups1 = {
        '평생학습 효과성 평균': [
            '문H2-1) 평생학습 참여가 삶의 질 향상에 얼마나 도움^ 아직 경험이 없다면 삶의 질 향상에 얼마나 도움이 될 것인지 생각하십니까?__1) 정신적 건강(정서적 안정감)',
            '문H2-1) 평생학습 참여가 삶의 질 향상에 얼마나 도움^ 아직 경험이 없다면 삶의 질 향상에 얼마나 도움이 될 것인지 생각하십니까?__2) 육체적 건강',
            '문H2-1) 평생학습 참여가 삶의 질 향상에 얼마나 도움^ 아직 경험이 없다면 삶의 질 향상에 얼마나 도움이 될 것인지 생각하십니까?__3) 사회참여 만족도',
            '문H2-1) 평생학습 참여가 삶의 질 향상에 얼마나 도움^ 아직 경험이 없다면 삶의 질 향상에 얼마나 도움이 될 것인지 생각하십니까?__4) 경제적 안정감'],
        '학습 지향 평균': [
            '문F1-5) 다음 학습 관련 질문에 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__교육훈련은 보다 나은 일상생활을 영위하는 데 도움을 준다.',
            '문F1-6) 다음 학습 관련 질문에 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__새로운 것을 배우는 것은 즐겁다.',
            '문F1-7) 다음 학습 관련 질문에 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__학습을 통해 자신감을 얻는다.',
            '문F1-8) 다음 학습 관련 질문에 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__성인학습자는 자신의 학습을 위해 무언가를 지불할 각오를 해야 한다.'],
        '직업관련 목표지향 평균': [
            '문F1-1) 다음 학습 관련 질문에 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__성인이 되어서도 지속적으로 학습을 하는 사람은 일자리를 잃을 가능성이 적다.',
            '문F1-2) 다음 학습 관련 질문에 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__성공적인 직장생활을 위해서는 지식과 기술을 끊임없이 향상시켜야 한다.',
            '문F1-3) 다음 학습 관련 질문에 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__고용주는 고용인들의 훈련을 책임져야 한다.']
    }

    # 각 그룹별 평균 계산
    for group_name, columns in col_groups1.items():
        df[group_name] = df[columns].mean(axis=1, skipna=True)

    # 각 그룹별 컬럼 리스트 딕셔너리로 정리
    col_groups2 = {
        '무형식학습(의존형) 참여': ['문B1-1) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__가족^ 친구 또는 직장동료^ 상사의 도움이나 조언을 통해 지식을 습득한 적이 있다', 
                      '문B1-5) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__학습을 목적으로 텔레비전^ 라디오 등을 활용해서 새로운 지식을 습득한 적이 있다', 
                      '문B1-6) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__책이나 전문잡지 등 인쇄매체를 활용해서 지식을 습득한 적이 있다'],
        '무형식학습(온라인) 참여': ['문B1-2) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__트위터^ 페이스북^ 카페^ 블로그^ 밴드 등을 활용해서 새로운 정보나 기술을 습득한 적이 있다', 
                      '문B1-3) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__유튜브(Youtube) 등을 활용해서 새로운 정보나 기술을 습득한 적이 있다', 
                      '문B1-4) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__인터넷 뉴스^ E-book 등 온라인매체를 활용해서 새로운 정보나 기술을 습득한 적이 있다',],
        '무형식학습 (암묵적 학습) 참여': ['문B1-7) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__역사적·자연적·산업적 장소를 방문해서 지식을 습득한 적이 있다',
                    '문B1-8) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__도서관 등을 방문해서 새로운 사실을 배운 적이 있다',
                    '문B1-9) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__축제^ 박람회^ 음악회 등에 참여해서 무언가를 새롭게 배우거나 깊이 있게 알게된 적이 있다',
                    '문B1-10) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__스포츠^ 등산 등 신체를 움직이는 활동에 참여해서 무언가를 새롭게 배우거나 깊이 있게 알게 된 적이 있다']
    }
    
    # 반복문으로 파생변수 생성 (1 → 1, 2 → 0 으로 변환 후 처리) #1있음, 2없음 #1참여안함,2참여함 
    for group_name, columns in col_groups2.items():
        df[group_name] = (df[columns].applymap(lambda x: 1 if x == 1 else 0)
                                     .any(axis=1)
                                     .astype(int))    

    cols_groups3 = {
        '사회참여': [
                    '문G1-1) 지난 한 해 동안 자원봉사 또는 재능기부를 한 적이 있습니까? 있다면^ 몇 회 정도 참여하셨습니까?__참여경험',
                    '문G1-2) 지난 한 해 동안 자선단체에 기부 또는 후원한 적이 있습니까? 있다면^ 몇 회 정도 참여하셨습니까?__참여경험',
                    '문G1-3) 지난 한 해 동안 동아리에서 활동한 적이 있습니까? 있다면^ 몇 회 정도 참여하셨습니까?__참여경험',
                    '문G1-4) 지난 한 해 동안 지역사회단체에 참여한 적이 있습니까? 있다면^ 몇 회 정도 참여하셨습니까?__참여경험'
        ]
    }
    
    # 1->0, 2->1 으로 변환 후, 파생변수 처리(1이 하나라도 있으면 1)
    for group_name, columns in cols_groups3.items():
        df[group_name] = (df[columns].applymap(lambda x: 2 if x == 1 else 0)
                                     .any(axis=1)
                                     .astype(int))
    return df

In [5]:
def other_variable(df):
    # 1:예 2:아니오 -> 1:예 0:아니오
    df['문D1) 귀하께서는 작년에 평생교육 프로그램 관련 정보를 접한 적이 있습니까?'] = df['문D1) 귀하께서는 작년에 평생교육 프로그램 관련 정보를 접한 적이 있습니까?'].map({1: 1, 2: 0})

    # 99(모른다) -> 5(중간 정도 만족) : 54개 값 이동
    df['문H1) 귀하께서는 전반적으로 현재의 생활에 얼마나 만족하십니까?'] = df['문H1) 귀하께서는 전반적으로 현재의 생활에 얼마나 만족하십니까?'].replace(99, 5)

    # 9(모름) -> Nan -> 0
    df[['직장의 규모', '근무기간']] = df[['직장의 규모', '근무기간']].replace(9, np.nan)

    df['문C1) 앞으로 참여하길 희망하는 프로그램은 무엇입니까? (중복응답)1'] = df['문C1) 앞으로 참여하길 희망하는 프로그램은 무엇입니까? (중복응답)1'].apply(reclassify_program)

    df['문C2) 귀하께서는 작년(교육이나 학습 프로그램의 참여여부와 관계없이)에 참여하고 싶었지만 참여하지 못했던 평생교육 프로그램이 있었습니까?'] = df['문C2) 귀하께서는 작년(교육이나 학습 프로그램의 참여여부와 관계없이)에 참여하고 싶었지만 참여하지 못했던 평생교육 프로그램이 있었습니까?'].map({1: 1, 2: 0})

    df['근로시간형태'] = df['근로시간형태'].replace(np.nan, 0)
    return df

In [6]:
# 전체 전처리 함수
def preprocessing_edu(file_path):
    # 데이터 불러오기
    df = pd.read_csv(file_path)
    del df['ID']
    del df['원가중치']
    
    # 년도 재정렬
    df = df.sort_values(by='key', ascending=False)
    # 인덱스 초기화
    df = df.reset_index(drop=True)
    
    # 컬럼명 변경
    df.columns = [
        col.replace('B1', 'A3') if '문B1' in col and '귀하께서 작년에 참여한 학습경험이 전반적으로' in col else col
        for col in df.columns]

    # DQ1. 22년과 18년도  value차이 조정
    df.loc[(df['key'] == 2018) & (df['DQ1. 귀하의 최종 학력은 어떻게 되십니까?__상태구분'] == 3), 'DQ1. 귀하의 최종 학력은 어떻게 되십니까?__상태구분'] = 5

    # 파생변수 생성
    df = derived_variable(df)

    # Y 변수 생성
    df = Y_variable(df)

    # 그 외 변수 조정
    df = other_variable(df)

    # Y값 분리
    Y = df[['비형식교육 참여여부']]
    df = df.drop(['비형식교육 참여여부'], axis=1)
      
    nan_col0 = ['직업관련 비형식교육 참여자', '고용형태', '직업', '직장의 규모', '근무기간', '취약계층', 
           'DQ9. 귀하의 현재 고용형태는 어떻게 되십니까?', 'DQ10. 귀하는 현재 어떤 직업에 종사하고 계십니까?', 
           'DQ11. 귀하가 현재 소속되어 있는 직장의 규모는 어느 정도입니까? (사업장^ 상시근로자 기준)', 'DQ1. 귀하의 최종 학력은 어떻게 되십니까?__상태구분', 
           '문C1) 앞으로 참여하길 희망하는 프로그램은 무엇입니까? (중복응답)1', 
           '문B1-3) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__유튜브(Youtube) 등을 활용해서 새로운 정보나 기술을 습득한 적이 있다', 
           '문B1-4) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__인터넷 뉴스^ E-book 등 온라인매체를 활용해서 새로운 정보나 기술을 습득한 적이 있다',
           '문G1-3) 지난 한 해 동안 동아리에서 활동한 적이 있습니까? 있다면^ 몇 회 정도 참여하셨습니까?__참여횟수', 'DQ3. 몇 명과 함께 지내고 있습니까?__명',
           'DQ12. 귀하의 현재 직장에서의 근무기간은 얼마나 되십니까?__년', 'DQ12. 귀하의 현재 직장에서의 근무기간은 얼마나 되십니까?__개월']

    # 범주형/연속형 컬럼의 NaN -> 0 변경
    df[nan_col0] = df[nan_col0].fillna(0)
  
    # 빈 문자열을 NaN으로 변환
    df.replace(' ', np.nan, inplace=True)
    # 50% 이상 결측값인 컬럼 삭제
    threshold = len(df) * 0.5
    df_cleaned = df.dropna(thresh=threshold, axis=1)
    # 단일 값만 존재하는 컬럼 삭제
    df_cleaned = df_cleaned.loc[:, df_cleaned.nunique() > 1]
    
    # 원래 데이터에서 분리했던 칼럼 다시 결합
    df_cleaned = pd.concat([df_cleaned, Y], axis=1)

    # 최종 데이터 저장
    #df_cleaned.to_csv('df_preprocessing_0415.csv', index=False)
    return df_cleaned

df_cleaned = preprocessing_edu("C:/Users/JONJI/EducationCompetition/df_merged.csv")
display(df_cleaned)

,문A1-1) 작년(21.01.01~21.12.31) (1) 학위(졸업장) 취득을 위한 교육 참여 경험 여부,문B1-1) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__가족^ 친구 또는 직장동료^ 상사의 도움이나 조언을 통해 지식을 습득한 적이 있다,문B1-2) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__트위터^ 페이스북^ 카페^ 블로그^ 밴드 등을 활용해서 새로운 정보나 기술을 습득한 적이 있다,문B1-3) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__유튜브(Youtube) 등을 활용해서 새로운 정보나 기술을 습득한 적이 있다,문B1-4) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__인터넷 뉴스^ E-book 등 온라인매체를 활용해서 새로운 정보나 기술을 습득한 적이 있다,문B1-5) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__학습을 목적으로 텔레비전^ 라디오 등을 활용해서 새로운 지식을 습득한 적이 있다,문B1-6) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__책이나 전문잡지 등 인쇄매체를 활용해서 지식을 습득한 적이 있다,문B1-7) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__역사적·자연적·산업적 장소를 방문해서 지식을 습득한 적이 있다,문B1-8) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__도서관 등을 방문해서 새로운 사실을 배운 적이 있다,문B1-9) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__축제^ 박람회^ 음악회 등에 참여해서 무언가를 새롭게 배우거나 깊이 있게 알게된 적이 있다,...,동시 참여자,key,평생학습 효과성 평균,학습 지향 평균,직업관련 목표지향 평균,무형식학습(의존형) 참여,무형식학습(온라인) 참여,무형식학습 (암묵적 학습) 참여,사회참여,비형식교육 참여여부
0,2,2,1,1.00,1.00,1,2,2,1,1,...,2,2022,3.75,4.00,3.67,1,1,1,1,1
1,2,1,2,1.00,2.00,2,2,2,2,2,...,2,2022,3.25,3.50,3.33,1,1,1,1,0
2,2,1,2,1.00,2.00,2,2,2,2,2,...,2,2022,3.50,3.75,3.67,1,1,0,1,0
3,2,1,2,1.00,1.00,2,2,2,2,2,...,2,2022,3.75,3.00,2.33,1,1,0,1,1
4,2,1,2,1.00,2.00,2,2,2,2,2,...,2,2022,3.25,3.75,3.33,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51364,2,2,1,0.00,0.00,2,2,2,2,2,...,2,2018,4.00,3.75,3.67,0,1,1,1,1
51365,2,2,1,0.00,0.00,2,2,2,2,1,...,2,2018,4.50,4.00,2.33,0,1,1,1,0
51366,2,2,1,0.00,0.00,2,2,2,2,2,...,2,2018,3.75,3.75,3.67,0,1,0,1,0
51367,2,1,1,0.00,0.00,1,2,2,2,2,...,2,2018,3.50,4.25,3.67,1,1,0,1,0


In [8]:
df_cleaned.columns

Index(['문A1-1) 작년(21.01.01~21.12.31) (1) 학위(졸업장) 취득을 위한 교육 참여 경험 여부',
       '문B1-1) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__가족^ 친구 또는 직장동료^ 상사의 도움이나 조언을 통해 지식을 습득한 적이 있다',
       '문B1-2) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__트위터^ 페이스북^ 카페^ 블로그^ 밴드 등을 활용해서 새로운 정보나 기술을 습득한 적이 있다',
       '문B1-3) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__유튜브(Youtube) 등을 활용해서 새로운 정보나 기술을 습득한 적이 있다',
       '문B1-4) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__인터넷 뉴스^ E-book 등 온라인매체를 활용해서 새로운 정보나 기술을 습득한 적이 있다',
       '문B1-5) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__학습을 목적으로 텔레비전^ 라디오 등을 활용해서 새로운 지식을 습득한 적이 있다',
       '문B1-6) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__책이나 전문잡지 등 인쇄매체를 활용해서 지식을 습득한 적이 있다',
       '문B1-7) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__역사적·자연적·산업적 장소를 방문해서 지식을 습득한 적이 있다',
       '문B1-8) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__도서관 등을 방문해서 새로운 사실을 배운 적이 있다',
       '문B1-9) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 있으십니까?__축제^ 박람회^ 음악회 등에 참여해서 무언가를 새롭게 배우거나 깊이 있게 알게된 적이 있다',
       '문B1-10) 귀하께서는 작년에 다음과 같은 학습에 참여해 본 적이 

In [ ]:
# 선택한 컬럼
selcted_cols = ['평생학습 효과성 평균', '학습 지향 평균', '직업관련 목표지향 평균', '무형식학습(의존형) 참여', 
                    '무형식학습(온라인) 참여', '무형식학습 (암묵적 학습) 참여', '사회참여', 
                    '문D1) 귀하께서는 작년에 평생교육 프로그램 관련 정보를 접한 적이 있습니까?', '문H1) 귀하께서는 전반적으로 현재의 생활에 얼마나 만족하십니까?',
                    '문C1) 앞으로 참여하길 희망하는 프로그램은 무엇입니까? (중복응답)1', '문C2) 귀하께서는 작년(교육이나 학습 프로그램의 참여여부와 관계없이)에 참여하고 싶었지만 참여하지 못했던 평생교육 프로그램이 있었습니까?', 
                    '문E1-1) 자신의 학습패턴에 관하여 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__나는 책이나 판서 수업 중심으로 공부하는 것이 익숙하고 잘된다.',
                    '문E1-2) 자신의 학습패턴에 관하여 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__나는 인터넷강의 또는 컴퓨터(전자책^ 태블릿 PC^스마트폰)를 활용한 학습이 익숙하고 잘된다.',
                    '문E1-3) 자신의 학습패턴에 관하여 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__나는 혼자서 공부하는 것이 익숙하고 좋다.',
                    '문E1-4) 자신의 학습패턴에 관하여 어느 정도 동의하시는지 동의 정도를 말씀해주시기 바랍니다.__나는 다른 사람들과 함께 공부 하는 것이 좋다.',
                    '성별', '연령별(1)', '학력별', '경활상태', '월 가구소득', '지역규모별', '부양가족 유무', '건강에 대한 자신감', '취업구분',
                    '근로시간형태', '직업', '직장의 규모', '근무기간', '형식교육 참여자', 'key', '비형식교육 참여여부',
        
    ]